In [1]:
import spotipy
import spotipy.util as util
#sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd
import time

In [37]:
keys

{'scope': 'user-library-read playlist-read-private user-read-currently-playing user-read-private playlist-read-collaborative',
 'cid': '8b8dc230afd94c0ab5a072c2cbca8701',
 'secret': '2b0489d1c9c14b30b71a26afb5fadba8',
 'username': 'claham510?si=eCCEfQ42QDKQceHjqms1TA'}

In [32]:
import json
with open('pass.json','r') as f:
    keys = json.load(f)['spotify_api']
    

In [44]:
def refresh_api():
    scope = keys['scope']
    cid =keys['cid'] 
    secret = keys['secret']
    username = keys['username']
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    token=util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri='https://google.com/')
    return sp
sp =refresh_api()

# Get Dataframe

In [129]:
#get the original dataframe

df = pd.read_csv('cf_dataset.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [130]:
df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [131]:
df2 = df.copy()
#filter out the unique ones that we are getting features on
df2.drop_duplicates('song_id',inplace=True)
len(df2)

10000

# Testing the queery out from spotipy

In [ ]:
test = sp.search(q='Stronger artist:Kanye West')

In [158]:
test['tracks']['items'][0]['uri']

'spotify:track:4fzsfWzRhPawzqhX8Qt9F3'

In [159]:
def create_queery_col(data):
    #this will give a more specific queery since albums are also taking into account
    data['test'] = data[['title', 'artist_name']].apply(lambda x: ' artist:'.join(x), axis=1)
    data['queery'] = data[['test', 'release']].apply(lambda x: ' album:'.join(x), axis=1)
    data.drop(columns='test',inplace=True)
    data.reset_index(drop=True,inplace=True)
    
    #some album queeries did not work since albums had been removed etc, so create another simpler queery
    #def filter_queery()
    data['filter_que'] = data[['title', 'artist_name']].apply(lambda x: ' artist:'.join(x), axis=1)
    
create_queery_col(df2)
df2['ids']=None #prep the id column
df2.head()

,user_id,song_id,listen_count,title,release,artist_name,year,queery,ids
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove artist:Jack Johnson album:Thicker Tha...,None
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas artist:Paco De Lucia album:Fla...,None
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger artist:Kanye West album:Graduation,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations artist:Jack Johnson album:In Be...,spotify:track:4rpNjW4HXxh8o5UwS7UoWL
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly artist:Foo Fighters album:There I...,spotify:track:5OQsiBsky2k2kDKy2bX2eT


# Call the api with our queeries

In [ ]:
def get_stuff(filters=False):
  if filters: #filters is the 2nd queery column, for those with less specific queeries
    for i,k in enumerate(df2['filter_que']):
      if df2.loc[i,'ids']==None:
        test = sp.search(q=k)
        try:
          sid = test['tracks']['items'][0]['uri']
          if i%100==0:
            print(i)
          #print(sid)
          df2.loc[i,'ids']= sid
        except:
          continue
      else:
        continue
  else:
    for i,k in enumerate(df2['queery']):
      
      test = sp.search(q=df2.loc[i,'queery'])
      try:
        sid = test['tracks']['items'][0]['uri']
        if i%100==0:
          print(i)
        #print(sid)
        df2.loc[i,'ids']= sid
      except:
        continue
#comment out these since they take forever to run
#get_stuff(filters=False) 
#get_stuff(filters=True)


In [ ]:
def get_songs(songs=False,earlystop=True): #if songs it creates a dataframe with all songs instead
       
    #song specific
    
    song_feature_list = ['danceability',
        'acousticness',
        'energy',
        'instrumentalness',
        'key',
        'liveness',
        'loudness',
        'mode',
        'tempo',
        'valence']   
    
    feats = {'name':[],
             'id':[],
             'popularity':[],
             'danceability':[],
        'acousticness':[],
        'energy':[],
        'instrumentalness':[],
        'key':[],
        'liveness':[],
        'loudness':[],
        'mode':[],
        'tempo':[],
        'valence':[]}
    
    if songs:
        feats['artists']=[]
        feats['album']=[]
        feats['explicit']=[]
    
    
    count = 0
    ii=0
    song = sp.
    playlists = sp.user_playlists('spotify', limit=50)
    
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            try:
                if earlystop and i>=250:
                    ii = 250
                    break
                
                metadata = sp.user_playlist('spotify', playlist_id=playlist['id'])                
                
                if songs:
                    if count%1000==0:
                        print(f"count: {count}")
                    for index, item in enumerate(metadata['tracks']['items']):
                        #get the song features
                        features= song_handler(item['track']['uri'],lists=False)
                        for k in song_feature_list:
                            feats[k].append(features[k])
                            
                        feats['name'].append(item['track']['name'])
                        feats['popularity'].append(item['track']['popularity'])
                        feats['explicit'].append(item['track']['explicit'])
                        feats['album'].append(item['track']['album']['name'])
                        feats['artists'].append(item['track']['artists'][0]['name'])
                        feats['id'].append(item['track']['uri'])
                        
                        
                        count+=1
                        if (len(feats['name'])!= len(feats['key'])):
                            print(f"lists not same lenghts for {feats['name']}")
                        
                        
                else: #for playlists
                    songs=[]
                    popularities = np.empty(len(metadata['tracks']['items']))
                    for index, item in enumerate(metadata['tracks']['items']): 
                        popularities[index] = item['track']['popularity']
                        songs.append(item['track']['id'])
                    #playlist specific appends
                    feats['popularity'].append(popularities.mean())
                    feats['name'].append(playlist['name'])
                    feats['id'].append(metadata['id'])
                    feats['followers'].append(metadata['followers']['total'])    
                    feats['tot_tracks'].append(playlist['tracks']['total'])
                    
                    features= song_handler(songs,lists=True) #getting the audio features
                    for k in feats:
                        feats[k].append(features[k])
                    
                print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
                
                          
            except:
                print("NO METADATA")
        
        if earlystop and ii==250:break
        
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
            break
    
    return feats

#datadict= create_dataset(songs=True,earlystop=True)

In [ ]:
#the rest of the data will not be recovered so drop these none ids
df2.dropna(axis=0,how='any',inplace=True)
df2.reset_index(inplace=True,drop=True)
df3.drop(columns='index',inplace=True)
df2.to_csv('full_cf_with_idss.csv')
len(df2)

In [111]:
df3 = pd.read_csv('full_cf_with_ids2.csv',index_col=0) 
df3.head()

,user_id,song_id,listen_count,title,release,artist_name,year,queery,ids,filter_que,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,Entre Dos Aguas artist:Paco De Lucia,0.458,0.933000,0.657,0.878000,4.0,0.0842,-12.020,0.0,102.450,0.914
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0,Stronger artist:Kanye West album:Graduation,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,Stronger artist:Kanye West,0.617,0.005640,0.717,0.000000,10.0,0.4080,-7.858,0.0,103.992,0.490
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,Constellations,In Between Dreams,Jack Johnson,2005.0,Constellations artist:Jack Johnson album:In Be...,spotify:track:4rpNjW4HXxh8o5UwS7UoWL,Constellations artist:Jack Johnson,0.705,0.476000,0.309,0.000038,1.0,0.1140,-12.596,1.0,122.039,0.426
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999.0,Learn To Fly artist:Foo Fighters album:There I...,spotify:track:5OQsiBsky2k2kDKy2bX2eT,Learn To Fly artist:Foo Fighters,0.465,0.000018,0.919,0.000021,4.0,0.2620,-4.025,1.0,135.997,0.537
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5.0,Apuesta Por El Rock 'N' Roll,Antología Audiovisual,Héroes del Silencio,2007.0,Apuesta Por El Rock 'N' Roll artist:Héroes del...,spotify:track:1ZlHDBGmSZ7w06NL7i60GR,Apuesta Por El Rock 'N' Roll artist:Héroes del...,0.444,0.273000,0.889,0.000278,7.0,0.1930,-3.756,1.0,169.907,0.633


In [58]:
for i,k in enumerate(df3['key']):
    stop=np.nan
    if k==np.nan:
        stop = i
        break
    else: continue
stop

nan

In [118]:
len(df3)

7762

In [150]:
stuff = sp.audio_features(df3.loc[x,'ids'])
stuff

[{'danceability': 0.626,
  'energy': 0.108,
  'key': 9,
  'loudness': -15.027,
  'mode': 1,
  'speechiness': 0.048,
  'acousticness': 0.832,
  'instrumentalness': 0,
  'liveness': 0.111,
  'valence': 0.195,
  'tempo': 139.379,
  'type': 'audio_features',
  'id': '57SgdCgjbJlEH2yPhJUTFy',
  'uri': 'spotify:track:57SgdCgjbJlEH2yPhJUTFy',
  'track_href': 'https://api.spotify.com/v1/tracks/57SgdCgjbJlEH2yPhJUTFy',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/57SgdCgjbJlEH2yPhJUTFy',
  'duration_ms': 259984,
  'time_signature': 4}]

In [24]:
    
def get_features(data,index,first_time=False):
        if first_time:
            data['danceability'] = None
            data['acousticness'] = None
            data['energy'] = None
            data['instrumentalness'] = None
            data['key'] = None
            data['liveness'] = None
            data['loudness'] = None
            data['mode'] = None
            data['tempo'] = None
            data['valence'] = None

        for count,k in enumerate(data.loc[index:,'ids']):
            
            if count%100==0:
                data.to_csv('full_cf_with_ids3.csv')
                print(count)
            stuff = sp.audio_features(k)
            i = index+count
            print(i)
            if i==len(data)-1:
                    print(f'end with {i}')
                    data.to_csv('full_cf_with_ids3.csv')
                    break
            try:
                
                data.loc[i,'danceability'] = stuff[0]['danceability'] 
                data.loc[i,'acousticness'] = stuff[0]['acousticness'] 
                data.loc[i,'energy'] = stuff[0]['energy'] 
                data.loc[i,'instrumentalness'] = stuff[0]['instrumentalness'] 
                data.loc[i,'key'] = stuff[0]['key'] 
                data.loc[i,'liveness'] = stuff[0]['liveness'] 
                data.loc[i,'loudness'] = stuff[0]['loudness'] 
                data.loc[i,'mode'] = stuff[0]['mode'] 
                data.loc[i,'tempo'] = stuff[0]['tempo'] 
                data.loc[i,'valence'] = stuff[0]['valence']

            except: #some dont work
                continue
            
                #data.drop(index=i,inplace=True)
            
                
                


In [16]:
sp =refresh_api()



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=8b8dc230afd94c0ab5a072c2cbca8701&response_type=code&redirect_uri=https%3A%2F%2Fgoogle.com%2F&scope=playlist-read-collaborative+playlist-read-private+user-library-read+user-read-currently-playing+user-read-private in your browser




Enter the URL you were redirected to:  https://www.google.com/?code=AQBIM-C6lDEUF0fXedBJvGM929GWisH3yOaOgBpKLIs5486m4WKNgKgEeR8URUnaIxN3b4WeSQZAeR6iuWPTZhwM-H8OGIfdGq8fGmBoAgle0sSOP4lmy4DoPAtBYO7ll4HqPZyy6hsadFbMvXzDocZMgli3PDu0NYhMz3CGgjviGA2oE2TqdOLa-cS-zgfRTLMu6ZnLEu_OkK20JAZN8QPj9wyHEKua3iY2x0Fl0ACdCvlarZilDzjEzWDUpi287jxIJ3aoWsQor9jZC4PhzqLodHetLgBVewkG8-pCCxHWD8qd9egIhgR3QjYwI_0ffOGho9-Qm6WWYBJwaQb0Qw


In [25]:
start = time.time()
get_features(df,index=0)
print(time.time()-start)

0
7700
7701
7702
7703
7704
7705
7706
7707
7708
7709
7710
7711
7712
7713
7714
7715
7716
7717
7718
7719
7720
7721
7722
7723
7724
7725
7726
7727
7728
7729
7730
7731
7732
7733
7734
7735
7736
7737
7738
7739
7740
7741
7742
7743
7744
7745
7746
7747
7748
7749
7750
7751
7752
7753
7754
7755
7756
7757
7758
7759
7760
7761
end with 7761
7.881559610366821


In [26]:
df = pd.read_csv('full_cf_with_ids3.csv',index_col=0)
df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,queery,ids,filter_que,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,Entre Dos Aguas artist:Paco De Lucia,0.458,0.933000,0.657,0.878000,4.0,0.0842,-12.020,0.0,102.450,0.914
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0,Stronger artist:Kanye West album:Graduation,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,Stronger artist:Kanye West,0.617,0.005640,0.717,0.000000,10.0,0.4080,-7.858,0.0,103.992,0.490
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,Constellations,In Between Dreams,Jack Johnson,2005.0,Constellations artist:Jack Johnson album:In Be...,spotify:track:4rpNjW4HXxh8o5UwS7UoWL,Constellations artist:Jack Johnson,0.705,0.476000,0.309,0.000038,1.0,0.1140,-12.596,1.0,122.039,0.426
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999.0,Learn To Fly artist:Foo Fighters album:There I...,spotify:track:5OQsiBsky2k2kDKy2bX2eT,Learn To Fly artist:Foo Fighters,0.465,0.000018,0.919,0.000021,4.0,0.2620,-4.025,1.0,135.997,0.537
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5.0,Apuesta Por El Rock 'N' Roll,Antología Audiovisual,Héroes del Silencio,2007.0,Apuesta Por El Rock 'N' Roll artist:Héroes del...,spotify:track:1ZlHDBGmSZ7w06NL7i60GR,Apuesta Por El Rock 'N' Roll artist:Héroes del...,0.444,0.273000,0.889,0.000278,7.0,0.1930,-3.756,1.0,169.907,0.633


In [30]:
#check for all values
df.loc[7761,:]

user_id                      695c0dac9bf302af160bb7ff16a171bec7fdea21
song_id                                            SOPBPHJ12AAF3B59B6
listen_count                                                        2
title               Baby [You've Got What It Takes] [with Sharon J...
release                                                    Crazy Love
artist_name                                             Michael Bublé
year                                                             2009
queery              Baby [You've Got What It Takes] [with Sharon J...
ids                              spotify:track:2zcZYRidtr3SYLsPlxWTz6
filter_que          Baby [You've Got What It Takes] [with Sharon J...
danceability                                                    0.632
acousticness                                                    0.227
energy                                                          0.708
instrumentalness                                             3.55e-05
key                 

In [32]:
df.loc[:,'mode'].isna().sum()

2

In [33]:
df.dropna(axis=0,inplace=True,how='any')
len(df)

7759

# Saving our dataframe to a csv 

In [34]:
df.to_csv('final_songs_features.csv')

# Combinding it with the user data

In [41]:
users = pd.read_csv('datasets/user_data_with_ratings.csv',index_col=0)
users.head()

C:\Users\johan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,song_id,listen_count,title,release,artist_name,year,raw_ratings
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,1


In [43]:
df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,queery,ids,filter_que,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,Entre Dos Aguas artist:Paco De Lucia,0.458,0.933000,0.657,0.878000,4.0,0.0842,-12.020,0.0,102.450,0.914
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0,Stronger artist:Kanye West album:Graduation,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,Stronger artist:Kanye West,0.617,0.005640,0.717,0.000000,10.0,0.4080,-7.858,0.0,103.992,0.490
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,Constellations,In Between Dreams,Jack Johnson,2005.0,Constellations artist:Jack Johnson album:In Be...,spotify:track:4rpNjW4HXxh8o5UwS7UoWL,Constellations artist:Jack Johnson,0.705,0.476000,0.309,0.000038,1.0,0.1140,-12.596,1.0,122.039,0.426
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999.0,Learn To Fly artist:Foo Fighters album:There I...,spotify:track:5OQsiBsky2k2kDKy2bX2eT,Learn To Fly artist:Foo Fighters,0.465,0.000018,0.919,0.000021,4.0,0.2620,-4.025,1.0,135.997,0.537
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5.0,Apuesta Por El Rock 'N' Roll,Antología Audiovisual,Héroes del Silencio,2007.0,Apuesta Por El Rock 'N' Roll artist:Héroes del...,spotify:track:1ZlHDBGmSZ7w06NL7i60GR,Apuesta Por El Rock 'N' Roll artist:Héroes del...,0.444,0.273000,0.889,0.000278,7.0,0.1930,-3.756,1.0,169.907,0.633


# Dropping a few columns since we only want the song info here for our 7000+ unique songs
## We can use the remaining songs that did get filtered out as unique non seen observations

In [45]:
df1 = df.copy()
df1.drop(columns=['user_id','listen_count','title','release','artist_name',
                  'year'],inplace=True)

In [46]:
df1.head()

,song_id,queery,ids,filter_que,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence
0,SOBBMDR12A8C13253B,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,Entre Dos Aguas artist:Paco De Lucia,0.458,0.933000,0.657,0.878000,4.0,0.0842,-12.020,0.0,102.450,0.914
1,SOBXHDL12A81C204C0,Stronger artist:Kanye West album:Graduation,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,Stronger artist:Kanye West,0.617,0.005640,0.717,0.000000,10.0,0.4080,-7.858,0.0,103.992,0.490
2,SOBYHAJ12A6701BF1D,Constellations artist:Jack Johnson album:In Be...,spotify:track:4rpNjW4HXxh8o5UwS7UoWL,Constellations artist:Jack Johnson,0.705,0.476000,0.309,0.000038,1.0,0.1140,-12.596,1.0,122.039,0.426
3,SODACBL12A8C13C273,Learn To Fly artist:Foo Fighters album:There I...,spotify:track:5OQsiBsky2k2kDKy2bX2eT,Learn To Fly artist:Foo Fighters,0.465,0.000018,0.919,0.000021,4.0,0.2620,-4.025,1.0,135.997,0.537
4,SODDNQT12A6D4F5F7E,Apuesta Por El Rock 'N' Roll artist:Héroes del...,spotify:track:1ZlHDBGmSZ7w06NL7i60GR,Apuesta Por El Rock 'N' Roll artist:Héroes del...,0.444,0.273000,0.889,0.000278,7.0,0.1930,-3.756,1.0,169.907,0.633


# Getting the complete dataset with features and user data!

In [48]:
#merge - inner - only keeps unique
#left - keep every row in left and set the right to nan's if there's no match, vice verse for right
# outer - all rows from both and matches where possible
result = pd.merge(users,df1,on='song_id',how='inner')

In [49]:
result.head()

,user_id,song_id,listen_count,title,release,artist_name,year,raw_ratings,queery,ids,...,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,2,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,...,0.458,0.933,0.657,0.878,4.0,0.0842,-12.02,0.0,102.45,0.914
1,930d2be6c85315d72cab9823ec0f7bfe7e477794,SOBBMDR12A8C13253B,1,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,1,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,...,0.458,0.933,0.657,0.878,4.0,0.0842,-12.02,0.0,102.45,0.914
2,41c12ce05f18b757d9257e5ad4f8303b4b8a8758,SOBBMDR12A8C13253B,66,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,20,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,...,0.458,0.933,0.657,0.878,4.0,0.0842,-12.02,0.0,102.45,0.914
3,f8611810514c93d57bb846aa0b5e0410d9730a14,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,2,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,...,0.458,0.933,0.657,0.878,4.0,0.0842,-12.02,0.0,102.45,0.914
4,5968a59e582f434a223b3786cd51c9f4690b38d4,SOBBMDR12A8C13253B,6,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,6,Entre Dos Aguas artist:Paco De Lucia album:Fla...,spotify:track:3Fgk17TfdjXWFroJy29pXH,...,0.458,0.933,0.657,0.878,4.0,0.0842,-12.02,0.0,102.45,0.914


In [51]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1550217 entries, 0 to 1550216
Data columns (total 21 columns):
user_id             1550217 non-null object
song_id             1550217 non-null object
listen_count        1550217 non-null int64
title               1550217 non-null object
release             1550217 non-null object
artist_name         1550217 non-null object
year                1550217 non-null int64
raw_ratings         1550217 non-null int64
queery              1550217 non-null object
ids                 1550217 non-null object
filter_que          1550217 non-null object
danceability        1550217 non-null float64
acousticness        1550217 non-null float64
energy              1550217 non-null float64
instrumentalness    1550217 non-null float64
key                 1550217 non-null float64
liveness            1550217 non-null float64
loudness            1550217 non-null float64
mode                1550217 non-null float64
tempo               1550217 non-null float64
v

In [53]:
result.to_csv(r'C:\Users\johan\OneDrive\Documents\thesis\datasets\full_df.csv')

In [ ]:
#Gameplan:
\begin{enumerate}Get distribution for each user per features, this combines to a multinomial Gausian

# random a number for each distribution 